### Restart and Run All Cells

In [1]:
import pandas as pd
from sqlalchemy import create_engine
engine = create_engine("sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3")
conlt = engine.connect()
year = 2021
quarter = 4

In [2]:
cols = 'name year quarter q_amt_c q_amt_p inc_profit percent'.split()

In [3]:
format_dict = {
                'q_amt':'{:,}','q_amt_c':'{:,}','q_amt_p':'{:,}','inc_profit':'{:,}',
                'yoy_gain':'{:,}','acc_gain':'{:,}',    
                'q_eps':'{:.4f}','y_eps':'{:.4f}','aq_eps':'{:.4f}','ay_eps':'{:.4f}',
                'percent':'{:.2f}%'
              }

In [4]:
sql = """
SELECT name,year,quarter,q_amt
FROM epss 
WHERE (year = %s AND quarter <= %s) 
OR (year = %s-1 AND quarter >= %s+1)
ORDER BY year DESC, quarter DESC"""
sql = sql % (year, quarter, year, quarter)
print(sql)


SELECT name,year,quarter,q_amt
FROM epss 
WHERE (year = 2021 AND quarter <= 4) 
OR (year = 2021-1 AND quarter >= 4+1)
ORDER BY year DESC, quarter DESC


In [5]:
dfc = pd.read_sql(sql, conlt)
dfc["Counter"] = 1
dfc_grp = dfc.groupby(["name"], as_index=False).sum()
dfc_grp = dfc_grp[dfc_grp["Counter"] == 4]
dfc_grp.head().style.format(format_dict)

,name,year,quarter,q_amt,Counter
0,2S,8084,10,"737,516",4
1,AAV,8084,10,"-6,647,486",4
2,ACE,8084,10,"1,505,681",4
3,ADVANC,8084,10,"26,922,146",4
5,AH,8084,10,"1,023,968",4


In [6]:
sql = """
SELECT name,year,quarter,q_amt
FROM epss 
WHERE (year = %s-1 AND quarter <= %s) 
OR (year = %s-2 AND quarter >= %s+1)
ORDER BY year DESC, quarter DESC
"""
sql = sql % (year, quarter, year, quarter)
print(sql)


SELECT name,year,quarter,q_amt
FROM epss 
WHERE (year = 2021-1 AND quarter <= 4) 
OR (year = 2021-2 AND quarter >= 4+1)
ORDER BY year DESC, quarter DESC



In [7]:
dfp = pd.read_sql(sql, conlt)
dfp["Counter"] = 1
dfp_grp = dfp.groupby(["name"], as_index=False).sum()
dfp_grp = dfp_grp[dfp_grp["Counter"] == 4]
dfp_grp.head().style.format(format_dict)

,name,year,quarter,q_amt,Counter
0,2S,8080,10,"410,165",4
1,AAV,8080,10,"-4,764,091",4
2,ACE,8080,10,"1,507,641",4
3,ADVANC,8080,10,"27,434,360",4
4,AEONTS,8080,10,"3,689,643",4


In [8]:
dfm = pd.merge(dfc_grp, dfp_grp, on="name", suffixes=(["_c", "_p"]), how="inner")
dfm["inc_profit"] = dfm["q_amt_c"] - dfm["q_amt_p"]
dfm["percent"] = round(dfm["inc_profit"] / abs(dfm["q_amt_p"]) * 100, 2)
dfm["year"] = year
dfm["quarter"] = "Q" + str(quarter)
df_percent = dfm[cols]
df_percent.head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
0,2S,2021,Q4,"737,516","410,165","327,351",79.81%
1,AAV,2021,Q4,"-6,647,486","-4,764,091","-1,883,395",-39.53%
2,ACE,2021,Q4,"1,505,681","1,507,641","-1,960",-0.13%
3,ADVANC,2021,Q4,"26,922,146","27,434,360","-512,214",-1.87%
4,AH,2021,Q4,"1,023,968","137,642","886,326",643.94%


In [9]:
sql = '''
DELETE FROM yr_profits 
WHERE year = %s AND quarter = "Q%s"'''
sql = sql % (year, quarter)
print(sql)

rp = conlt.execute(sql)
rp.rowcount


DELETE FROM yr_profits 
WHERE year = 2021 AND quarter = "Q4"


303

In [10]:
sql = "SELECT name, id FROM tickers"
tickers = pd.read_sql(sql, conlt)
df_ins = pd.merge(df_percent, tickers, on="name", how="inner")
rcds = df_ins.values.tolist()
len(rcds)

307

In [11]:
#for rcd in rcds:
#    print(rcd)

In [12]:
sql = """
INSERT INTO yr_profits (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id) 
VALUES (?, ?, ?, ?, ?, ?, ?, ?)"""
print(sql)


INSERT INTO yr_profits (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id) 
VALUES (?, ?, ?, ?, ?, ?, ?, ?)


In [13]:
for rcd in rcds:
    conlt.execute(sql, rcd)

### End of loop

In [14]:
criteria_1 = df_ins.q_amt_c > 440_000
df_ins.loc[criteria_1, cols].head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
0,2S,2021,Q4,"737,516","410,165","327,351",79.81%
2,ACE,2021,Q4,"1,505,681","1,507,641","-1,960",-0.13%
3,ADVANC,2021,Q4,"26,922,146","27,434,360","-512,214",-1.87%
4,AH,2021,Q4,"1,023,968","137,642","886,326",643.94%
6,AIMIRT,2021,Q4,"487,775","413,822","73,953",17.87%


In [15]:
criteria_2 = df_ins.q_amt_p > 400_000
df_ins.loc[criteria_2, cols].head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
0,2S,2021,Q4,"737,516","410,165","327,351",79.81%
2,ACE,2021,Q4,"1,505,681","1,507,641","-1,960",-0.13%
3,ADVANC,2021,Q4,"26,922,146","27,434,360","-512,214",-1.87%
5,AIE,2021,Q4,"423,622","488,517","-64,895",-13.28%
6,AIMIRT,2021,Q4,"487,775","413,822","73,953",17.87%


In [16]:
criteria_3 = df_ins.percent > 10.00
df_ins.loc[criteria_3, cols].head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
0,2S,2021,Q4,"737,516","410,165","327,351",79.81%
4,AH,2021,Q4,"1,023,968","137,642","886,326",643.94%
6,AIMIRT,2021,Q4,"487,775","413,822","73,953",17.87%
7,AIT,2021,Q4,"527,125","394,271","132,854",33.70%
12,AMARIN,2021,Q4,"313,113","170,669","142,444",83.46%


In [17]:
final_criteria = criteria_1 & criteria_2 & criteria_3
df_ins.loc[final_criteria, cols].sort_values(by=["percent"], ascending=[False]).head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
124,IVL,2021,Q4,"26,288,027","2,414,276","23,873,751",988.86%
195,RCL,2021,Q4,"17,972,751","1,744,794","16,227,957",930.08%
30,BCH,2021,Q4,"6,846,002","1,229,405","5,616,597",456.85%
56,CKP,2021,Q4,"2,179,013","404,714","1,774,299",438.41%
53,CHG,2021,Q4,"4,204,072","876,617","3,327,455",379.58%


In [18]:
df_ins.loc[final_criteria, cols].sort_values(by=["name"], ascending=[True]).head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
0,2S,2021,Q4,"737,516","410,165","327,351",79.81%
6,AIMIRT,2021,Q4,"487,775","413,822","73,953",17.87%
13,AMATA,2021,Q4,"1,402,287","1,103,439","298,848",27.08%
20,ASIAN,2021,Q4,"1,044,316","818,056","226,260",27.66%
21,ASK,2021,Q4,"1,202,804","883,064","319,740",36.21%


In [19]:
df_ins.loc[final_criteria, cols].sort_values(by=["name"], ascending=[True]).head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
0,2S,2021,Q4,"737,516","410,165","327,351",79.81%
6,AIMIRT,2021,Q4,"487,775","413,822","73,953",17.87%
13,AMATA,2021,Q4,"1,402,287","1,103,439","298,848",27.08%
20,ASIAN,2021,Q4,"1,044,316","818,056","226,260",27.66%
21,ASK,2021,Q4,"1,202,804","883,064","319,740",36.21%
